In [1]:
import joblib
import numpy as np
import pandas as pd

preprocessing_object = joblib.load("/data/foundation_model/data/preprocessing_objects_20250615.pkl.z")
# print(preprocessing_object)

all_data = (
    pd.read_parquet("/data/foundation_model/data/qc_ac_te_mp_dos_reformat_20250615.pd.parquet").drop(
        index=preprocessing_object["dropped_idx"]
    )
    # .reset_index(drop=True)
)
all_data.head(3)

test_data = (
    all_data[all_data["split"] == "test"]
    #  .reset_index(drop=True)
)
test_data.head(3)


desc_trans = pd.read_parquet("/data/foundation_model/data/qc_ac_te_mp_dos_composition_desc_trans_20250615.pd.parquet")


/data/foundation_model/.venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/data/foundation_model/.venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator PowerTransformer from version 1.7.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/data/foundation_model/.venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.7.0 when using v

In [6]:
test_data.columns

Index(['formula', 'Material type', 'Space group', 'composition', 'elements',
       'Band gap', 'Density', 'Efermi', 'Final energy per atom',
       'Formation energy per atom', 'Total magnetization', 'Volume',
       'Dielectric total', 'Dielectric ionic', 'Dielectric electronic',
       'Electrical resistivity', 'Electrical resistivity (T/K)',
       'Power factor', 'Power factor (T/K)', 'Seebeck coefficient',
       'Seebeck coefficient (T/K)', 'Thermal conductivity',
       'Thermal conductivity (T/K)', 'ZT', 'ZT (T/K)',
       'Magnetic susceptibility', 'Magnetic susceptibility (T/K)',
       'Material type (label)', 'Space group (label)', 'Band gap (normalized)',
       'Density (normalized)', 'Efermi (normalized)',
       'Final energy per atom (normalized)',
       'Formation energy per atom (normalized)',
       'Total magnetization (normalized)', 'Volume (normalized)',
       'Dielectric total (normalized)', 'Dielectric ionic (normalized)',
       'Dielectric electronic (norm

In [2]:
def swap_train_val_split(split, swap_ratio=0.1, random_seed=None, train_ratio: float = 1.0):
    split = split.copy()
    train_idx = split[split == "train"].index
    val_idx = split[split == "val"].index

    # 先交换
    n_swap = int(min(len(train_idx), len(val_idx)) * swap_ratio)
    if n_swap > 0:
        rng = np.random.default_rng(random_seed)
        swap_train = rng.choice(train_idx, n_swap, replace=False)
        swap_val = rng.choice(val_idx, n_swap, replace=False)
        split.loc[swap_train] = "val"
        split.loc[swap_val] = "train"

    # 再采样train
    train_idx = split[split == "train"].index
    if train_ratio < 1.0:
        rng = np.random.default_rng(random_seed)
        n_train = round(len(train_idx) * train_ratio)
        sampled_train_idx = rng.choice(train_idx, n_train, replace=False)
        # 其余train直接丢弃
        drop_idx = train_idx.difference(sampled_train_idx)
        split.loc[drop_idx] = np.nan

    return split.dropna()


In [3]:
_ = [
    "Band gap",  # 0
    "Density",  # 1
    "Efermi",  # 2
    "Final energy per atom",  # 3
    "Formation energy per atom",  # 4
    "Total magnetization",  # 5
    "Volume",  # 6
    "Dielectric total",  # 7
    "Dielectric ionic",  # 8
    "Dielectric electronic",  # 9
]


In [4]:
import json
from datetime import datetime
from pathlib import Path

from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

N_TRY = 10

for prop_name in _:  # 0~9
    # Experiment 1: Use Fourier features
    print("training for property:", prop_name)

    for ratio in [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]:
        base_dir = Path(f"logs/regression/{prop_name}/{datetime.now().strftime('%m%d_%H%M')}_r{ratio}")

        scaler = preprocessing_object[f"{prop_name.lower().replace(' ', '_')}_scaler"]

        # mask = all_data[f"{prop_name} (normalized)"].notnull()
        mask = all_data[f"{prop_name}"].notnull()
        org_splits = all_data.loc[mask, "split"]
        # y_true = all_data.loc[mask, f"{prop_name} (normalized)"]
        prop = all_data.loc[mask, f"{prop_name}"]

        if prop_name in ["Dielectric total", "Dielectric ionic", "Dielectric electronic"]:
            prop = np.log(prop + 1e-8)  # log transform

        for n_try in range(N_TRY):
            # 1. Setup logging directory
            version = f"trial_{n_try + 1}"
            save_dir = base_dir / "predictions" / version
            save_dir.mkdir(parents=True, exist_ok=True)

            splits = swap_train_val_split(org_splits, swap_ratio=0.5, random_seed=None, train_ratio=ratio)
            splits.to_csv(f"{save_dir}/data_split.csv")

            X_train = desc_trans.loc[splits[splits == "train"].index]
            y_train = prop.loc[splits[splits == "train"].index]
            X_test = desc_trans.loc[splits[splits == "test"].index]
            y_test = prop.loc[splits[splits == "test"].index]

            rf = RandomForestRegressor(
                n_estimators=300, random_state=n_try, bootstrap=True, max_features="sqrt", n_jobs=60
            )
            rf = rf.fit(X_train, y_train)
            y_pred, y_true = rf.predict(X_test), y_test.values
            y_fit_pred, y_fit_true = rf.predict(X_train), y_train.values

            # 保存预测结果
            results = pd.concat(
                [
                    pd.DataFrame({"y_true": y_fit_true, "y_pred": y_fit_pred, "label": "train"}, index=X_train.index),
                    pd.DataFrame({"y_true": y_true, "y_pred": y_pred, "label": "test"}, index=X_test.index),
                ]
            )
            results.to_parquet(save_dir / "rf_predictions.parquet")
            results.to_csv(save_dir / "rf_predictions.csv")

            # 保存模型
            joblib.dump(rf, save_dir / "rf_model.pkl.z")

            # 评估
            fig, ax = plt.subplots(figsize=(5, 5), dpi=150)

            # inverse
            # y_true = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
            # y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
            # y_fit_true = scaler.inverse_transform(y_train.values.reshape(-1, 1)).flatten()
            # y_fit_pred = scaler.inverse_transform(y_fit_pred.reshape(-1, 1)).flatten()

            # inverse log
            # if prop_name in ["Dielectric total", "Dielectric ionic", "Dielectric electronic"]:
            #     y_true = np.exp(y_true) - 1e-8
            #     y_pred = np.exp(y_pred) - 1e-8
            #     y_fit_true = np.exp(y_fit_true) - 1e-8
            #     y_fit_pred = np.exp(y_fit_pred) - 1e-8

            # scatter plot
            ax.scatter(y_true, y_pred, alpha=0.1, label="Train")
            ax.scatter(y_fit_true, y_fit_pred, alpha=0.5, label="Test")
            ax.set_xlabel("Observation")
            ax.set_ylabel("Prediction")
            ax.set_title(
                f"{prop_name} {'(log) ' if prop_name in ['Dielectric total', 'Dielectric ionic', 'Dielectric electronic'] else ''}"
            )
            ax.plot(
                [min(y_true.min(), y_fit_true.min()), max(y_true.max(), y_fit_true.max())],
                [min(y_true.min(), y_fit_true.min()), max(y_true.max(), y_fit_true.max())],
                "r--",
            )
            mae = mean_absolute_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            ax.legend()
            ax.text(
                0.05,
                0.95,
                f"MAE={mae:.3f}\nR²={r2:.3f}",
                transform=ax.transAxes,
                verticalalignment="top",
                bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"),
            )

            metrics = {"mae": float(mae), "r2": float(r2)}
            split_counts = splits.value_counts().to_dict()
            metrics.update(split_counts)
            with open(f"{save_dir}/metrics.json", "w") as f:
                json.dump(metrics, f, indent=2)
            fig.savefig(f"{save_dir}/scatter.png", bbox_inches="tight")
            plt.cla()
            plt.clf()

training for property: Band gap


/tmp/ipykernel_1929582/97079514.py:64: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(5, 5), dpi=150)


training for property: Density
training for property: Efermi
training for property: Final energy per atom
training for property: Formation energy per atom
training for property: Total magnetization
training for property: Volume
training for property: Dielectric total
training for property: Dielectric ionic
training for property: Dielectric electronic


<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

...

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

In [7]:
preprocessing_object.keys()

dict_keys(['dropped_idx', 'composition_desc_scaler', 'material_type_label_encoder', 'space_group_label_encoder', 'band_gap_scaler', 'density_scaler', 'efermi_scaler', 'final_energy_per_atom_scaler', 'formation_energy_per_atom_scaler', 'total_magnetization_scaler', 'volume_scaler', 'dielectric_constant_scaler', 'thermal_conductivity_scaler', 'electrical_resistivity_scaler', 'power_factor_scaler', 'seebeck_coefficient_scaler', 'zt_scaler', 'magnetic_susceptibility_scaler', 'dielectric_total_scaler', 'dielectric_ionic_scaler', 'dielectric_electronic_scaler'])

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

N_TRY = 10
prop_name = "Material type"
# Experiment 1: Use Fourier features
print("training for property:", prop_name)

for ratio in [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]:
    base_dir = Path(f"logs/classification/{prop_name}/{datetime.now().strftime('%m%d_%H%M')}_r{ratio}")

    mask = all_data[f"{prop_name}"].notnull()
    org_splits = all_data.loc[mask, "split"]
    prop = all_data.loc[mask, f"{prop_name}"]

    for n_try in range(N_TRY):
        # 1. Setup logging directory
        version = f"trial_{n_try + 1}"
        save_dir = base_dir / "predictions" / version
        save_dir.mkdir(parents=True, exist_ok=True)

        splits = swap_train_val_split(org_splits, swap_ratio=0.5, random_seed=None, train_ratio=ratio)
        splits.to_csv(f"{save_dir}/data_split.csv")

        X_train = desc_trans.loc[splits[splits == "train"].index]
        y_train = prop.loc[splits[splits == "train"].index]
        X_test = desc_trans.loc[splits[splits == "test"].index]
        y_test = prop.loc[splits[splits == "test"].index]

        # 编码类别
        le = preprocessing_object["material_type_label_encoder"]
        y_train_enc = le.transform(y_train)
        y_test_enc = le.transform(y_test)

        # 训练随机森林分类器
        clf = RandomForestClassifier(
            n_estimators=300, random_state=n_try, bootstrap=True, max_features="sqrt", n_jobs=60
        )
        clf = clf.fit(X_train, y_train_enc)
        y_pred_enc = clf.predict(X_test)
        y_fit_pred_enc = clf.predict(X_train)
        y_pred, y_true = le.inverse_transform(y_pred_enc), y_test.values
        y_fit_pred, y_fit_true = le.inverse_transform(y_fit_pred_enc), y_train.values

        # 保存预测结果
        results = pd.concat(
            [
                pd.DataFrame({"y_true": y_fit_true, "y_pred": y_fit_pred, "label": "train"}, index=X_train.index),
                pd.DataFrame({"y_true": y_true, "y_pred": y_pred, "label": "test"}, index=X_test.index),
            ]
        )
        results.to_parquet(save_dir / "clf_predictions.parquet")
        results.to_csv(save_dir / "clf_predictions.csv")

        # 保存模型
        joblib.dump(clf, save_dir / "clf_model.pkl.z")

        test_report = classification_report(y_test, y_pred, target_names=le.classes_, digits=4, output_dict=True)
        train_report = classification_report(
            y_fit_true, y_fit_pred, target_names=le.classes_, digits=4, output_dict=True
        )

        # 混淆矩阵
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(21, 8), sharey=True, dpi=130)
        plt.subplots_adjust(wspace=0.05)

        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred, labels=le.classes_)
        hm1 = sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=le.classes_,
            yticklabels=le.classes_,
            ax=ax1,
            annot_kws={"size": 13},
        )
        ax1.set_xlabel("Predicted", fontsize=18)
        ax1.set_ylabel("True", fontsize=18)
        ax1.set_title(f"{prop_name}", fontsize=18)
        ax1.tick_params(axis="both", labelsize=13)
        cbar1 = hm1.collections[0].colorbar
        cbar1.ax.tick_params(labelsize=13)

        # Normalized confusion matrix
        cm_norm = confusion_matrix(y_test, y_pred, labels=le.classes_, normalize="true")
        hm2 = sns.heatmap(
            cm_norm,
            annot=True,
            fmt=".2f",
            cmap="Blues",
            xticklabels=le.classes_,
            yticklabels=le.classes_,
            ax=ax2,
            annot_kws={"size": 13},
        )
        ax2.set_xlabel("Predicted", fontsize=18)
        ax2.set_title(f"{prop_name} (Normalized)", fontsize=18)
        ax2.tick_params(axis="both", labelsize=13)
        cbar2 = hm2.collections[0].colorbar
        cbar2.ax.tick_params(labelsize=13)

        fig.savefig(f"{save_dir}/test_cm.png", bbox_inches="tight")
        plt.cla()
        plt.clf()

        # 分类指标
        print(classification_report(y_true, y_pred, target_names=le.classes_, digits=4, output_dict=True))

        # 混淆矩阵
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(21, 8), sharey=True, dpi=150)
        plt.subplots_adjust(wspace=0.05)

        # Confusion matrix
        cm = confusion_matrix(y_fit_true, y_fit_pred, labels=le.classes_)
        hm1 = sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=le.classes_,
            yticklabels=le.classes_,
            ax=ax1,
            annot_kws={"size": 15},
        )
        ax1.set_xlabel("Predicted", fontsize=18)
        ax1.set_ylabel("True", fontsize=18)
        ax1.set_title(f"{prop_name}", fontsize=18)
        ax1.tick_params(axis="both", labelsize=13)
        cbar1 = hm1.collections[0].colorbar
        cbar1.ax.tick_params(labelsize=13)

        # Normalized confusion matrix
        cm_norm = confusion_matrix(y_fit_true, y_fit_pred, labels=le.classes_, normalize="true")
        hm2 = sns.heatmap(
            cm_norm,
            annot=True,
            fmt=".2f",
            cmap="Blues",
            xticklabels=le.classes_,
            yticklabels=le.classes_,
            ax=ax2,
            annot_kws={"size": 13},
        )
        ax2.set_xlabel("Predicted", fontsize=18)
        ax2.set_title(f"{prop_name} (Normalized)", fontsize=18)
        ax2.tick_params(axis="both", labelsize=13)
        cbar2 = hm2.collections[0].colorbar
        cbar2.ax.tick_params(labelsize=13)

        fig.savefig(f"{save_dir}/train_cm.png", bbox_inches="tight")
        plt.cla()
        plt.clf()

        print(classification_report(y_fit_true, y_fit_pred, target_names=le.classes_, digits=4, output_dict=True))

        metrics = {"train": train_report, "test": test_report}
        with open(f"{save_dir}/metrics.json", "w") as f:
            json.dump(metrics, f, indent=2)

        break
    break

training for property: Material type
{'DAC': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1.0}, 'DQC': {'precision': 0.75, 'recall': 1.0, 'f1-score': 0.8571428571428571, 'support': 3.0}, 'IAC': {'precision': 0.8461538461538461, 'recall': 0.9166666666666666, 'f1-score': 0.88, 'support': 24.0}, 'IQC': {'precision': 0.8518518518518519, 'recall': 0.8214285714285714, 'f1-score': 0.8363636363636363, 'support': 28.0}, 'others': {'precision': 0.9995887594242632, 'recall': 0.9991778569471088, 'f1-score': 0.9993832659494278, 'support': 7298.0}, 'accuracy': 0.998232254555344, 'macro avg': {'precision': 0.7895188914859922, 'recall': 0.9474546190084695, 'f1-score': 0.8479112852245176, 'support': 7354.0}, 'weighted avg': {'precision': 0.9983557669890967, 'recall': 0.998232254555344, 'f1-score': 0.998269697029535, 'support': 7354.0}}
{'DAC': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 8.0}, 'DQC': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'su

<Figure size 2730x1040 with 0 Axes>

<Figure size 3150x1200 with 0 Axes>